# dash-cam Simulator

This python notebook serves the purpose of extracting the results of the dash-cam simulator 

In [1]:
import glob
import numpy as np
import os
import random
from simulator import *
from reads import Reads

#### Define the data directory

In [2]:
datadir = "data/"
kmer_size = 32
os.makedirs("data/", exist_ok=True)

#### Define the parallel simulator of dash-cam, and add references

In [3]:
parallel_search = ParallelSearch()
for genome_filepath in glob.glob(datadir + "/*.fna"):
    parallel_search.buildBlock(
        block_name=genome_filepath.split("/")[-1].split(".")[0],
        genome_file=genome_filepath
    )

#### Define read getter and list all available sequencer platforms

In [4]:
reads = Reads(datadir="data/", read_length=kmer_size)
platforms = reads.getPlatforms()

#### Generate results for the different platforms as a function of the dash-cam Hamming Distance toleration threshold

Note: The cell run might take a while.

In [5]:
for platform in platforms:
    parallel_search.recordResults(f"results/{platform}-threshold-varies.csv")
    for threshold in range(1):
        for genome_file in glob.glob(datadir + "/*.fna"):
            parallel_search.search(
                patterns=reads.getKmers(
                    platform=platform,
                    genome_filepath=genome_file,
                    kmer_size=kmer_size
                ),
                threshold=threshold,
                true_genome=genome_file.split("/")[-1].split(".")[0]
            )
    parallel_search.stopRecording()

#### Generate results for the accuracy as a function of the progressing time

In [ ]:
#### Generate time results
parallel_search.recordResults(f"results/time.csv")
for time in np.linspace(0, 150, 50):
    for genome_file in glob.glob(datadir + "/*.fna"):
        parallel_search.search(
            patterns=reads.getReads(
                platform=platform,
                genome_filepath=datadir + genome_file
            ),
            time=time,
            true_genome=genome_file.split("/")[-1].split(".")[0]
        )
parallel_search.stopRecording()

#### Generate results for the accuracy as a function of the varrying discharge rate

In [ ]:
parallel_search.recordResults(f"results/discharge-rate.csv")
for discharge_rate in np.linspace(0, 1, 50):
    for genome_file in glob.glob(datadir + "/*.fna"):
        parallel_search.search(
            patterns=reads.getReads(
                platform=platform,
                genome_filepath=datadir + genome_file
            ),
            discharge_rate=discharge_rate,
            true_genome=genome_file.split("/")[-1].split(".")[0]
        )
parallel_search.stopRecording()

In [11]:
reads = Reads("kraken2-data", read_length=150)

In [27]:
kraken2_db = "kraken2-data/dash-cam-db/"
kraken_output = "kraken-out.txt"

results = {}
for platform in reads.getPlatforms():
    for genome_filepath in glob.glob("data/*.fna"):
        genome_name = genome_filepath.split("/")[-1].split(".")[0]
        results[(platform, genome_name)] = {
            "tp": 0,
            "fn": 0,
            "fp": 0
        }
        for read_file in reads.getReadFiles(platform, genome_filepath=genome_filepath):
            os.system(f"kraken2/bin/kraken2 --db {kraken2_db} {read_file} --use-names --output {kraken_output}")
            with open(kraken_output, "r") as f:
                lines = f.read().split("\n")
                for line in lines:
                    if len(line) > 0:
                        results[(platform, genome_name)]["tp"] += line[0] == "C"
                        results[(platform, genome_name)]["fn"] += line[0] == "U"
os.system(f"rm -rf {kraken_output}")


Loading database information... done.
1990 sequences (0.30 Mbp) processed in 0.012s (9569.6 Kseq/m, 1435.44 Mbp/m).
  1990 sequences classified (100.00%)
  0 sequences unclassified (0.00%)
Loading database information... done.
1050 sequences (0.16 Mbp) processed in 0.006s (10994.8 Kseq/m, 1649.21 Mbp/m).
  1050 sequences classified (100.00%)
  0 sequences unclassified (0.00%)
Loading database information... done.
700 sequences (0.10 Mbp) processed in 0.004s (9593.4 Kseq/m, 1439.01 Mbp/m).
  699 sequences classified (99.86%)
  1 sequences unclassified (0.14%)
Loading database information... done.
850 sequences (0.13 Mbp) processed in 0.005s (10357.4 Kseq/m, 1553.61 Mbp/m).
  850 sequences classified (100.00%)
  0 sequences unclassified (0.00%)
Loading database information... done.
1200 sequences (0.18 Mbp) processed in 0.009s (8269.2 Kseq/m, 1240.38 Mbp/m).
  1200 sequences classified (100.00%)
  0 sequences unclassified (0.00%)
Loading database information... done.
1994 sequences (0.30

In [28]:
with open("results/kraken2.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["platform", "organism", "tp", "fn", "fp"])
    for platform, organism in results:
        writer.writerow([
            platform,
            organism,
            results[(platform, organism)]["tp"],
            results[(platform, organism)]["fn"],
            results[(platform, organism)]["fp"]])